In [6]:
from pathlib import Path
from matplotlib import pyplot as plt
import ast
import pysam 
import numpy as np
from Bio import SeqIO
import time 
import matplotlib.patches as patches
import matplotlib.cm as cm
import itertools
from tabulate import tabulate
import csv
import random

np.set_printoptions(threshold=np.inf)
min_quality_score = 8



assembly_ = open("/private/groups/migalab/dan/reference/hg002v1.0.1.fasta", "r")

start_time = time.time()

#Load the reference genome and make it into a dictionary 
fasta_sequences = SeqIO.parse(assembly_, "fasta")
assembly={}
for fasta in fasta_sequences:
    name, sequence = fasta.id, str(fasta.seq)
    assembly[name] = sequence

#Make a dictionary for all the chromosomes and their corresponding sequence length 
assembly_sequence_length = {}    
for chromosome in assembly:
    assembly_sequence_length[chromosome] = len(assembly[chromosome])
    
end_time = time.time()
elapsed_time = end_time - start_time
print (elapsed_time, "seconds")
assembly_.close()



63.14178776741028 seconds


In [7]:
CDR_regions = '/private/groups/migalab/dan/data_analysis/young_old_analysis/HG002_DiMeLo_CENPA_youngpassage.hmmCDR_only_CDR_Dan_certified.bed'
output_dir = Path('/private/groups/migalab/dan/data_analysis/HG002_figure2/b_revised_multi_region')
CENPA_AS_bam_file = '/private/groups/migalab/dan/06_11_24_R1041_UL_DiMeLo_CENPAyoung_1/20240611_1126_1H_PAW33460_814408d8/pod5/06_11_24_R1041_UL_DiMeLo_CENPAyoung_1_5mA_6mC_winnowmap_MD.bam'
CENPC_AS_bam_file = '/private/groups/migalab/dan/01_09_24_R1041_DiMeLoAdaptive_CENPC/01_09_24_R1041_DiMeLoAdaptive_CENPC/01_09_24_R1041_DiMeLoAdaptive_CENPC/20240109_1200_6B_PAS52674_0adbae11/pod5_pass/explicit/01_09_24_R1041_DiMeLoAdaptive_CENPC_5mC_6mA_winnowmap_sorted_MD.bam'
H3K9me3_AS_bam_file = '/private/groups/migalab/dan/08_05_24_R1041_ULadapt_Dimelo_H3K9ME3/08_05_24_R1041_ULadapt_Dimelo_H3K4ME3/08_05_24_R1041_ULadapt_Dimelo_H3K4ME3_1/20240805_1148_1F_PAU87705_0451cc00/pod5/08_05_24_R1041_ULadapt_Dimelo_H3K4ME3_mA_mC_winnowmap_sorted_MD.bam'
ref_genome_file = Path('/private/groups/migalab/dan/reference/hg002v1.0.1.fasta')


In [8]:
''' here in the code, I am formulating the CDR regions and listing the CDRs in each and every chromosome'''
input_file = '/private/groups/migalab/dan/data_analysis/young_old_analysis/HG002_DiMeLo_CENPA_youngpassage.hmmCDR_only_CDR_Dan_certified.bed'
CDR_dict = {}
with open(input_file, 'r') as infile:  
    for i in infile:
        chr_num = i.split('\t')[0]
        CDR_start = i.split('\t')[1]
        CDR_end = i.split('\t')[2].split('\n')[0]
        if chr_num not in CDR_dict:
            CDR_dict[chr_num] = [[CDR_start ,CDR_end]]
        elif chr_num in CDR_dict:  
            CDR_dict[chr_num].append ([CDR_start ,CDR_end])





In [9]:
'''Based on the CDR regions, I am obtaining CDR adjacent regions in the same format as the CDR data set above'''
CDR_adjacent = {}
for chromosome in CDR_dict: 
    CDR_adjacent[chromosome] =[]
    for CDR in CDR_dict[chromosome]: 
        #print (CDR) 
        CDR_adjacent_left_space = [int(CDR[0]) - 1001, int(CDR[0]) - 1]
        CDR_adjacent_right_space = [int(CDR[1]) + 1, int(CDR[1]) + 1001]
        
        CDR_adjacent[chromosome].append (CDR_adjacent_left_space)
        CDR_adjacent[chromosome].append (CDR_adjacent_right_space)
        
    

In [10]:
'''The purpose of the code here is to make an active array where the chromosome name is
the dictionary key. The active array blocks are in lists inside the key.
'''
import os
input_file = '/private/groups/migalab/dan/data_analysis/alpha_bed/hg002v1.0.fasta.manualAlpha.cenSat_H1L_merged.bed'
active_dict = {}
with open(input_file, 'r') as infile:  
    num = 0 
    previous_chr_num = ""
    previous_parental_status = ""
    for i in infile:
        chr_num = i.split ('_')[0]
        parental_status = i.split ('_')[1][0:8]
        if chr_num == previous_chr_num and parental_status !=previous_parental_status:
            num = 0 
        elif chr_num == previous_chr_num and parental_status ==previous_parental_status:
            pass 
        else: 
            num = 0 
        active = i.split ()
        if active[0] not in active_dict:
            
            active_dict[active[0]] = [[int(active[1]) ,int(active[2])]]
        else:
            active_dict[active[0]].append([int(active[1]) ,int(active[2])])
        previous_chr_num = chr_num 
        previous_parental_status = parental_status

In [11]:
none_CDR_active = {}
for chromosome in CDR_dict: 
    none_CDR_active[chromosome] = [] 
    active_start = active_dict[chromosome][0][0]
    active_end = active_dict[chromosome][0][1]
    CDR_start_position = int (CDR_dict[chromosome][0][0]) - 1 
    CDR_end_position = int (CDR_dict[chromosome][-1][1]) + 1
    none_CDR_active[chromosome].append ([active_start, CDR_start_position])
    for coordinate in range(len(CDR_dict[chromosome]) - 1):
        # Generate in-between coordinates
        
        start_of_next = int (CDR_dict[chromosome][coordinate + 1][0]) - 1 
        end_of_current = int (CDR_dict[chromosome][coordinate][1]) + 1 
        # Ensure there's no overlap and the next start is greater than the current end
        if int(start_of_next) > int(end_of_current):
            none_CDR_active[chromosome].append([end_of_current, start_of_next])
    
none_CDR_active[ chromosome].append ([CDR_end_position, active_end])

In [12]:
def count_regions(chromosome_dict):
    """
    Counts the number of regions (small lists) for each chromosome in the dictionary.
    
    Parameters:
        chromosome_dict (dict): Dictionary where keys are chromosome names and values are lists of regions.
    
    Returns:
        dict: Dictionary with chromosome names as keys and the count of regions as values.
    """
    region_counts = {}
    for chromosome, regions in chromosome_dict.items():
        region_counts[chromosome] = len(regions)
    return region_counts



In [13]:
def split_into_segments(chromosome_dict, segment_size=5000):
    """
    Splits the ranges in the chromosome dictionary into smaller segments of a specified size.
    Ignores segments smaller than the specified size.
    
    Parameters:
        chromosome_dict (dict): Dictionary with chromosome keys and list of ranges.
        segment_size (int): Size of each segment to create (default: 5000).
        
    Returns:
        dict: Dictionary with the same keys but ranges split into smaller segments.
    """
    segmented_dict = {}

    for chromosome, ranges in chromosome_dict.items():
        segmented_ranges = []
        for start_str, end_str in ranges:
            #print (start_str, end_str)
            start = int(start_str)
            end = int(end_str)
            # Generate segments
            while start + segment_size - 1 < end:  # Only create segments of full size
                segment_end = start + segment_size - 1
                segmented_ranges.append([str(start), str(segment_end)])
                start += segment_size  # Move to the next segment
        segmented_dict[chromosome] = segmented_ranges

    return segmented_dict



# Call the function
segmented_chromosome_CDR_dict = split_into_segments(CDR_dict)
segmented_chromosome_non_CDR_dict = split_into_segments(none_CDR_active)

print (count_regions(segmented_chromosome_CDR_dict))


def preserve_first_x_regions(chromosome_dict, x):
    """
    Preserves only the first x small lists (regions) for each chromosome in the dictionary.
    
    Parameters:
        chromosome_dict (dict): Dictionary with chromosome names as keys and lists of regions as values.
        x (int): The number of regions to preserve for each chromosome.
        
    Returns:
        dict: Dictionary with only the first x regions for each chromosome.
    """
    limited_dict = {}
    for chromosome, regions in chromosome_dict.items():
        limited_dict[chromosome] = regions[:x]  # Preserve only the first x regions
    return limited_dict

trimmed_segmented_chromosome_non_CDR_dict = preserve_first_x_regions(segmented_chromosome_non_CDR_dict, 15)
print (count_regions(trimmed_segmented_chromosome_non_CDR_dict))


{'chr10_MATERNAL': 15, 'chr10_PATERNAL': 13, 'chr11_MATERNAL': 20, 'chr11_PATERNAL': 19, 'chr12_MATERNAL': 17, 'chr12_PATERNAL': 16, 'chr13_MATERNAL': 15, 'chr13_PATERNAL': 12, 'chr14_MATERNAL': 16, 'chr14_PATERNAL': 17, 'chr15_MATERNAL': 14, 'chr15_PATERNAL': 14, 'chr16_MATERNAL': 13, 'chr16_PATERNAL': 18, 'chr17_MATERNAL': 16, 'chr17_PATERNAL': 19, 'chr18_MATERNAL': 13, 'chr18_PATERNAL': 13, 'chr19_MATERNAL': 17, 'chr19_PATERNAL': 18, 'chr1_MATERNAL': 12, 'chr1_PATERNAL': 11, 'chr20_MATERNAL': 13, 'chr20_PATERNAL': 15, 'chr21_MATERNAL': 13, 'chr21_PATERNAL': 13, 'chr22_MATERNAL': 10, 'chr22_PATERNAL': 15, 'chr2_MATERNAL': 15, 'chr2_PATERNAL': 20, 'chr3_MATERNAL': 17, 'chr3_PATERNAL': 15, 'chr4_MATERNAL': 11, 'chr4_PATERNAL': 14, 'chr5_MATERNAL': 16, 'chr5_PATERNAL': 15, 'chr6_MATERNAL': 18, 'chr6_PATERNAL': 12, 'chr7_MATERNAL': 16, 'chr7_PATERNAL': 15, 'chr8_MATERNAL': 16, 'chr8_PATERNAL': 15, 'chr9_MATERNAL': 13, 'chr9_PATERNAL': 14, 'chrX_MATERNAL': 15}
{'chr10_MATERNAL': 15, 'chr1

In [14]:
'''
The idea of this function is to isolate the the desired regions (here in the function, it is called the subset) in the mod 
numpy array without dashes(insertions)'''

def mod_subset_producing_step (mod_no_dash,alignment_dash,target_start_no_dash,target_end_no_dash):
    #mod_no_dash = is the numpy array of the mod without any insertions and deletions
    # alignment_dash = is the alignment sequence with the dashes in it 
    # target_start = it's the subset starting position WITHOUT the dashes!!! 


    # Create a mask to identify non-dash positions
    mask = [char != '-' for char in alignment_dash]

    # Generate cumulative counts only for True values in the mask
    cumulative_counts = list(itertools.accumulate(mask))


    
    # Create the final indexes list
    indexes = [count - 1 if is_non_dash else '-' for count, is_non_dash in zip(cumulative_counts, mask)]



    target_start_dash = indexes.index (target_start_no_dash)

        
    try:
        target_end_dash = indexes.index (target_end_no_dash)
    except ValueError: 
        target_end_dash = indexes[-1]




    #obtain dashed alignment 
    alignment_dash_sequence_pre_subset = alignment_dash[0:target_start_dash]
    alignment_dash_sequence_subset = alignment_dash[target_start_dash:target_end_dash]

    #create no dash alignment 
    alignment_no_dash_sequence_pre_subset = alignment_dash_sequence_pre_subset.replace("-","")
    alignment_no_dash_sequence_subset = alignment_dash_sequence_subset.replace("-","")

    subset_no_dash_start = len(alignment_no_dash_sequence_pre_subset)
    subset_no_dash_end = subset_no_dash_start + len(alignment_no_dash_sequence_subset)

    #make mod_no_dash alignment
    mod_subset = mod_no_dash[subset_no_dash_start:subset_no_dash_end]

    return mod_subset





In [15]:
''' 
The input of the function is a dictionary in the format of 'chromosome':[[start,end],[start,end]] 
'''
def region_read_mA_density_calculator (chromosome_coordinates,threshold,mod_tag,filtering_val): 
    read_region_check = {}
    read_CDR_region_distribution = {}


    #get each chromosome
    for chr_name in chromosome_coordinates:

        for region in chromosome_coordinates[chr_name]:
            region_base = 0 

            region_start_index = int(region[0])
            region_end_index = int(region[1])

            
            for read in bamfile.fetch(chr_name,region_start_index,region_end_index):
                chromosome = bamfile.get_reference_name(read.reference_id)
                read_name = read.query_name 
                read_start_position = read.reference_start
                read_end_position = read.reference_end
                
                    
                
                #make an if statement to check a specific read front, middle, end regions 
                #setting read start, end, density, length variables 
                    

                #Get the starting and ending positions of the reads 
                
                read_density = 0 

                
                #Get sequence information which shows deletions and insertions 
                sequence = read.get_aligned_pairs(matches_only=False, with_seq = True)


                #make |a numpy of the sequence length which eliminates the deletion
                read_sequence_insertion_included = ''
                genomic_alignment_sequence_deletion_mistach_included = ''
                
                for item in sequence:
                    if item[0] is None:
                        read_sequence_insertion_included+='-'
                    elif item[1] is None:
                        genomic_alignment_sequence_deletion_mistach_included += '-'
                    else: 
                        read_sequence_insertion_included+=item[2]
                        genomic_alignment_sequence_deletion_mistach_included +=item[2]

                
                #take sequence length excluding insertions 
                read_sequence_insertion_included = read_sequence_insertion_included.upper()
                genomic_alignment_sequence_deletion_mistach_included = genomic_alignment_sequence_deletion_mistach_included.upper()
                
                genomic_alignment_sequence_deletion_mistach_included_mask = np.array(
                [char != '-' for char in genomic_alignment_sequence_deletion_mistach_included])

                insertions = read_sequence_insertion_included.count ("-")
                no_insertion_no_deletion_sequence_length = len(read_sequence_insertion_included)

                #make a mod np array with the length of the read length
                mod=read.modified_bases_forward
                
                #make a mod score with its original length 
                mod_score = np.zeros(len(genomic_alignment_sequence_deletion_mistach_included),)
                try:
                    if mod_tag == 'A':
                        for indices, values in mod[('A', 0, 'a')]:
                            mod_score[indices] = values
                        mod_score = mod_score[genomic_alignment_sequence_deletion_mistach_included_mask]
                        


                    elif mod_tag == 'CG':
                        for indices, values in mod[('C', 0, 'm')]:
                            mod_score[indices] = values
                        mod_score = mod_score[genomic_alignment_sequence_deletion_mistach_included_mask]
                    
                    if read.is_reverse:
                            mod_score = mod_score[::-1]


                # No mod would return KeyError 
                except KeyError:
                    continue

                mod_score = np.where(mod_score < filtering_val, 0, mod_score)



                # if the regions are longer than the reads 
                if (region_end_index - region_start_index) > (read_end_position - read_start_position):
                    # if the reads are inside the region
                    if (region_end_index >= read_end_position) and (region_start_index <= read_start_position): 
                        mod_start = 0
                        mod_end = len(read_sequence_insertion_included)
                    
                    # if the reads cover the later part of the region
                    elif (region_end_index < read_end_position) and (region_start_index > read_start_position): 
                        mod_start = 0
                        mod_end = no_insertion_no_deletion_sequence_length - read_end_position - region_end_index

                    # if the reads cover the starting part of the region 
                    elif (region_end_index > read_end_position) and (region_start_index > read_start_position): 
                        mod_start = region_start_index - read_start_position 
                        mod_end = no_insertion_no_deletion_sequence_length

                        
                
                # if the reads are longer than the region selected 
                elif (region_end_index - region_start_index) <= (read_end_position - read_start_position):
                    # scenario 1: when the defined region is inside the read
                    if (read_start_position <= region_start_index) and (read_end_position >= region_end_index):
                        mod_start = region_start_index - read_start_position 
                        mod_end = region_end_index - read_start_position

                    # scenario 3: when the defined region covers a bit of the end of the read
                    elif (read_end_position < region_end_index) and (read_end_position > region_start_index):
                        mod_start = region_start_index - read_start_position
                        mod_end = no_insertion_no_deletion_sequence_length

                    # scenario 2: when the defined region covers a bit of the beginning of the read
                    elif (read_start_position > region_start_index) and (read_start_position < region_end_index):
                        mod_start = 0
                        mod_end = region_end_index - read_start_position 

                #use the defined starting and ending positons in the region to subset mod numpy
                if (region_start_index - read_start_position) > (no_insertion_no_deletion_sequence_length - insertions):
                    continue
                try:
                    trimmed_mod_score = mod_subset_producing_step (mod_score,read_sequence_insertion_included,mod_start,mod_end)
                except ValueError:
                    continue
                
            
                region_base += (mod_end - mod_start)
                #removing all the zeros 
                mod_no_zeros = trimmed_mod_score[trimmed_mod_score != 0]
                m_mod_tag = len (mod_no_zeros)
                

                #Getting the total amount of As in the subsetted region of the sequence 
                total_mod_tag = read_sequence_insertion_included[mod_start:mod_end].count(mod_tag)
            
            
                #calculate read density
                try:
                    read_density = m_mod_tag / total_mod_tag
                    if read_name in read_CDR_region_distribution:
                        read_CDR_region_distribution[read_name].append ([m_mod_tag,total_mod_tag,region_start_index])
                    else: 
                        read_CDR_region_distribution[read_name] = [[m_mod_tag,total_mod_tag,region_start_index]]
                    
                except ZeroDivisionError:
                    pass

                

                if mod_tag == "A":

                    read_CDR_location_density = {}
                    read_CDR_location_density[region_start_index] = read_density
                    if read_name+"*"+chromosome in read_region_check: 
                        read_region_check[read_name+"*"+chromosome].append (read_CDR_location_density)
                    else: 
                        read_region_check[read_name+"*"+chromosome]= [read_CDR_location_density]

                    
                    
                elif mod_tag == "CG":
                    read_CDR_location_density = {}
                    read_CDR_location_density[region_start_index] = read_density
                    if read_name+"*"+chromosome in read_region_check:
                        read_region_check[read_name+"*"+chromosome].append (read_CDR_location_density)
                    else:
                        read_region_check[read_name+"*"+chromosome]= [read_CDR_location_density]



    return  read_region_check, read_CDR_region_distribution

                                



    

In [ ]:
bamfile = pysam.AlignmentFile(CENPC_AS_bam_file, "rb")

In [14]:


CENPC_reads_non_CDR_dict_mC, CENPC_reads_non_CDR_distribution_mC = region_read_mA_density_calculator (trimmed_segmented_chromosome_non_CDR_dict,0,"CG",210)

In [15]:
CENPC_reads_CDR_dict_mC, CENPC_reads_CDR_distribution_mC = region_read_mA_density_calculator (segmented_chromosome_CDR_dict,0,"CG",210)

In [18]:
with open('/private/groups/migalab/dan/gini_heatmap/CENPC_reads_non_CDR_dict_mC.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in CENPC_reads_non_CDR_dict_mC.items():
        writer.writerow([key] + value)
with open('/private/groups/migalab/dan/gini_heatmap/CENPC_reads_non_CDR_distribution_dict_mC.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in CENPC_reads_non_CDR_distribution_mC.items():
        writer.writerow([key] + value)

In [19]:
with open('/private/groups/migalab/dan/gini_heatmap/CENPC_reads_CDR_dict_mC.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in CENPC_reads_CDR_dict_mC.items():
        writer.writerow([key] + value)
with open('/private/groups/migalab/dan/gini_heatmap/CENPC_reads_CDR_distribution_dict_mC.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in CENPC_reads_CDR_distribution_mC.items():
        writer.writerow([key] + value)

In [16]:
CENPC_reads_non_CDR_dict_mA, CENPC_reads_non_CDR_distribution_dict_mA = region_read_mA_density_calculator (trimmed_segmented_chromosome_non_CDR_dict,0,"A",230)

In [17]:
CENPC_reads_CDR_dict_mA, CENPC_reads_CDR_distributiondict_mA = region_read_mA_density_calculator (segmented_chromosome_CDR_dict,0,"A",230)

In [20]:
with open('/private/groups/migalab/dan/gini_heatmap/CENPC_reads_non_CDR_dict_mA.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in CENPC_reads_non_CDR_dict_mA.items():
        writer.writerow([key] + value)
with open('/private/groups/migalab/dan/gini_heatmap/CENPC_reads_non_CDR_distribution_dict_mA.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in CENPC_reads_non_CDR_distribution_dict_mA.items():
        writer.writerow([key] + value)

In [21]:
with open('/private/groups/migalab/dan/gini_heatmap/CENPC_reads_CDR_dict_mA.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in CENPC_reads_CDR_dict_mA.items():
        writer.writerow([key] + value)
with open('/private/groups/migalab/dan/gini_heatmap/CENPC_reads_CDR_distribution_dict_mA.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in CENPC_reads_CDR_distributiondict_mA.items():
        writer.writerow([key] + value)

KeyboardInterrupt: 

In [16]:
bamfile = pysam.AlignmentFile(H3K9me3_AS_bam_file, "rb")

In [27]:
H3K9_reads_non_CDR_dict_mA, H3K9_reads_non_CDR_distribution_mA = region_read_mA_density_calculator (trimmed_segmented_chromosome_non_CDR_dict,0,"A",230)

In [28]:
with open('/private/groups/migalab/dan/gini_heatmap/H3K9_reads_non_CDR_dict_mA.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in H3K9_reads_non_CDR_dict_mA.items():
        writer.writerow([key] + value)
with open('/private/groups/migalab/dan/gini_heatmap/H3K9_reads_non_CDR_distribution_dict_mA.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in H3K9_reads_non_CDR_distribution_mA.items():
        writer.writerow([key] + value)

In [17]:
H3K9_reads_CDR_dict_mA, H3K9_reads_CDR_distribution_mA = region_read_mA_density_calculator (segmented_chromosome_CDR_dict,0,"A",230)

In [18]:
H3K9_reads_CDR_dict_mC, H3K9_reads_CDR_distribution_mC = region_read_mA_density_calculator (segmented_chromosome_CDR_dict,0,"CG",210)

In [30]:
with open('/private/groups/migalab/dan/gini_heatmap/H3K9_reads_CDR_dict_mA.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in H3K9_reads_CDR_dict_mA.items():
        writer.writerow([key] + value)
with open('/private/groups/migalab/dan/gini_heatmap/H3K9_reads_CDR_distribution_dict_mA.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in H3K9_reads_CDR_distribution_mA.items():
        writer.writerow([key] + value)

In [32]:
bamfile = pysam.AlignmentFile(CENPC_AS_bam_file, "rb")

In [33]:
CENPC_reads_CDR_dict_mC, CENPC_reads_CDR_distribution_dict_mC = region_read_mA_density_calculator (segmented_chromosome_CDR_dict,0,"CG",210)

In [25]:
CENPC_reads_non_CDR_dict_mA, CENPC_reads_non_CDR_distribution_dict_mA = region_read_mA_density_calculator (trimmed_segmented_chromosome_non_CDR_dict,0,"A",230)



In [27]:
with open('/private/groups/migalab/dan/CENPC_reads_non_CDR_dict_mA.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in CENPC_reads_non_CDR_dict_mA.items():
        writer.writerow([key] + value)

with open('/private/groups/migalab/dan/CENPC_reads_non_CDR_distribution_dict_mA.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in CENPC_reads_non_CDR_distribution_dict_mA.items():
        writer.writerow([key] + value)

In [26]:
with open('/private/groups/migalab/dan/CENPC_reads_CDR_dict_mA.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in CENPC_reads_CDR_dict_mA.items():
        writer.writerow([key] + value)

with open('/private/groups/migalab/dan/CENPC_reads_CDR_distribution_dict_mA.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in CENPC_reads_CDR_distribution_dict_mA.items():
        writer.writerow([key] + value)

In [20]:
bamfile = pysam.AlignmentFile(CENPA_AS_bam_file, "rb")

In [19]:
CENPA_reads_CDR_dict_mA, CENPA_reads_CDR_distributiondict_mA = region_read_mA_density_calculator (segmented_chromosome_CDR_dict,0,"A",230)

In [21]:
with open('/private/groups/migalab/dan/CENPA_reads_CDR_dict_mA.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in CENPA_reads_CDR_dict_mA.items():
        writer.writerow([key] + value)

with open('/private/groups/migalab/dan/CENPA_reads_CDR_distribution_dict_mA.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in CENPA_reads_CDR_distributiondict_mA.items():
        writer.writerow([key] + value)

In [14]:
H3K9_reads_non_CDR_dict_mA, H3K9_reads_non_CDR_dict_mA = region_read_mA_density_calculator (trimmed_segmented_chromosome_non_CDR_dict,0,"A",230)


In [12]:
# Save the dictionary to a CSV file
with open('/private/groups/migalab/dan/reads_non_CDR_dict_mA', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in reads_non_CDR_dict_mA.items():
        writer.writerow([key] + value)

with open('/private/groups/migalab/dan/reads_non_CDR_mA_distribution_dict_mA', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in reads_non_CDR_mA_distribution_dict_mA.items():
        writer.writerow([key] + value)

NameError: name 'region_read_mA_density_calculator' is not defined

In [16]:

def parse_csv_to_dataframe(cdr_path, non_cdr_path):
    def load_dict_from_csv(file_path):
        with open(file_path, 'r') as csv_file:
            reader = csv.reader(csv_file)
            return {rows[0]: [ast.literal_eval(item) for item in rows[1:]] for rows in reader}
    
    reads_CDR_mC = load_dict_from_csv(cdr_path)
    reads_non_CDR_mC = load_dict_from_csv(non_cdr_path)
    
    return reads_CDR_mC, reads_non_CDR_mC

def convert_to_chromosome_dict(reads_dict):
    """
    Convert the input dictionary to a new structure where chromosome names are keys,
    and the values are lists of dictionaries containing the read names and their content.

    Parameters:
        reads_dict (dict): Input dictionary with keys in the format 'read*chromosome'.

    Returns:
        dict: Dictionary with chromosome names as keys and lists of dictionaries as values.
    """
    chromosome_dict = {}

    for key, content in reads_dict.items():
        # Split the key into read name and chromosome name
        read_name, chromosome = key.split('*')

        # Initialize the chromosome key if not already present
        if chromosome not in chromosome_dict:
            chromosome_dict[chromosome] = {}

        # Append the read and its content as a dictionary to the chromosome key
        chromosome_dict[chromosome][key] = content

    return chromosome_dict

def parse_and_convert(cdr_path, non_cdr_path):
    reads_CDR_mC, reads_non_CDR_mC = parse_csv_to_dataframe(cdr_path, non_cdr_path)
    
    chromosome_dict_reads_CDR_mC = convert_to_chromosome_dict(reads_CDR_mC)
    chromosome_dict_reads_non_CDR_mC = convert_to_chromosome_dict(reads_non_CDR_mC)
    
    return chromosome_dict_reads_CDR_mC, chromosome_dict_reads_non_CDR_mC


In [13]:
CENPA_CDR_dict_mC = '/private/groups/migalab/dan/gini_heatmap/CENPA_reads_CDR_dict_mC.csv'
CENPA_non_CDR_dict_mC = '/private/groups/migalab/dan/gini_heatmap/CENPA_reads_non_CDR_dict_mC.csv'
CENPA_chromosome_dict_reads_CDR_mC, CENPA_chromosome_dict_reads_non_CDR_mC= parse_and_convert(CENPA_CDR_dict_mC, CENPA_non_CDR_dict_mC)

CENPC_CDR_dict_mC = '/private/groups/migalab/dan/gini_heatmap/CENPC_reads_CDR_dict_mC.csv'
CENPC_non_CDR_dict_mC = '/private/groups/migalab/dan/gini_heatmap/CENPC_reads_non_CDR_dict_mC.csv'
CENPC_chromosome_dict_reads_CDR_mC, CENPC_chromosome_dict_reads_non_CDR_mC= parse_and_convert(CENPC_CDR_dict_mC, CENPC_non_CDR_dict_mC)

H3K9_CDR_dict_mC = '/private/groups/migalab/dan/gini_heatmap/H3K9_reads_CDR_dict_mC.csv'
H3K9_non_CDR_dict_mC = '/private/groups/migalab/dan/gini_heatmap/H3K9_reads_non_CDR_dict_mC.csv'
H3K9_chromosome_dict_reads_CDR_mC, H3K9_chromosome_dict_reads_non_CDR_mC= parse_and_convert(H3K9_CDR_dict_mC, H3K9_non_CDR_dict_mC)

In [10]:
def aggregate_density_by_chromosome(big_dict):
    """
    Aggregates density values for each chromosome from a nested dictionary.

    Parameters:
        big_dict (dict): A nested dictionary where keys are in the format 'UniqueReadID*ChromosomeName'
                         and values are lists of secondary dictionaries with starting positions as keys
                         and density values as values.

    Returns:
        dict: A dictionary where each chromosome name is a key, and the values are lists of aggregated density values.
    """
    # Create an empty dictionary to hold the aggregated densities for each chromosome
    chromosome_density_dict = {}

    # Iterate over the big dictionary
    for big_key, secondary_list in big_dict.items():
        # Extract the chromosome name from the big key
        chrom_name = big_key.split("*")[1]
        
        # Initialize the chromosome key in the new dictionary if not already present
        if chrom_name not in chromosome_density_dict:
            chromosome_density_dict[chrom_name] = []
        
        # Iterate over the list of secondary dictionaries
        for small_dict in secondary_list:
            # Add all density values to the corresponding chromosome key
            for density in small_dict.values():
                chromosome_density_dict[chrom_name].append(density)
    
    return chromosome_density_dict
CV_mA_chrom__non_CDR_result = aggregate_density_by_chromosome(reads_non_CDR_mC)
CV_mA_chrom_CDR_result = aggregate_density_by_chromosome(reads_CDR_mC)

In [21]:

def calculate_cv_between_chromosomes(chromosome_density_dict):
    """
    Calculates the Coefficient of Variation (CV) between chromosomes
    based on the average density value for each chromosome.

    Parameters:
        chromosome_density_dict (dict): A dictionary where each chromosome name is a key, 
                                        and the values are lists of aggregated density values.

    Returns:
        None: Prints the CV between chromosomes.
    """
    # Calculate the average density for each chromosome
    average_densities = []

    for chrom, densities in chromosome_density_dict.items():
        # Skip empty lists to avoid division by zero
        if len(densities) == 0:
            print(f"{chrom}: No data available.")
            continue
        
        # Calculate the average density for the chromosome
        avg_density = np.mean(densities)
        average_densities.append(avg_density)

    # Convert to a numpy array for easy calculation
    average_densities_array = np.array(average_densities)

    # Calculate mean and standard deviation of the average densities
    mean_avg_density = np.mean(average_densities_array)
    std_dev_avg_density = np.std(average_densities_array)

    # Calculate Coefficient of Variation (CV = SD / Mean)
    if mean_avg_density != 0:
        cv = std_dev_avg_density / mean_avg_density
        print(f"\nCoefficient of Variation (CV) between chromosomes = {cv:.4f}")


print ("CV for non CDR ")
calculate_cv_between_chromosomes(CV_mA_chrom__non_CDR_result)

print ("CV for CDR ")
calculate_cv_between_chromosomes(CV_mA_chrom_CDR_result)

CV for non CDR 

Coefficient of Variation (CV) between chromosomes = 0.0849
CV for CDR 

Coefficient of Variation (CV) between chromosomes = 0.1594


In [17]:
from sklearn.metrics import confusion_matrix


def extract_and_find_optimal_threshold(reads_non_CDR_dict_mA, reads_spanning_dict_mA):
    """
    Combined function to extract densities, label them, and find the optimal threshold based on Gini impurity.

    Parameters:
        reads_non_CDR_dict_mA (dict): Dictionary for the non-CDR group.
        reads_spanning_dict_mA (dict): Dictionary for the spanning group.

    Returns:
        best_threshold (float): The threshold with the lowest Gini impurity.
        best_gini (float): The Gini impurity at the best threshold.
        purity_info (dict): Dictionary containing purity percentages of left and right labels.
        y_pred (list): Predicted labels based on the best threshold.
    """
    def extract_densities(data_dict, class_label):
        extracted_data = []
        for value_list in data_dict.values():
            for density_dict in value_list:
                for _, density in density_dict.items():
                    extracted_data.append((density, class_label))
        return extracted_data

    def calculate_gini_impurity(subset):
        total_samples = len(subset)
        if total_samples == 0:
            return 0
        class_counts = np.bincount(subset, minlength=2)
        probabilities = class_counts / total_samples
        return 1 - np.sum(probabilities ** 2)

    # Extract training data
    training_data_class_1 = extract_densities(reads_non_CDR_dict_mA, class_label=1)
    training_data_class_0 = extract_densities(reads_spanning_dict_mA, class_label=0)
    training_data = training_data_class_1 + training_data_class_0

    # Sort data by density
    training_data = sorted(training_data, key=lambda x: x[0])
    densities = [d[0] for d in training_data]
    labels = [d[1] for d in training_data]

    midpoints = [(densities[i] + densities[i + 1]) / 2 for i in range(len(densities) - 1)]

    best_threshold = None
    best_gini = float('inf')
    purity_info = {}

    for threshold in midpoints:
        left_labels = [label for density, label in zip(densities, labels) if density <= threshold]
        right_labels = [label for density, label in zip(densities, labels) if density > threshold]

        gini_left = calculate_gini_impurity(left_labels)
        gini_right = calculate_gini_impurity(right_labels)

        total_samples = len(labels)
        weighted_gini = (len(left_labels) / total_samples) * gini_left + (len(right_labels) / total_samples) * gini_right

        if weighted_gini < best_gini:
            best_gini = weighted_gini
            best_threshold = threshold

            left_class_counts = np.bincount(left_labels, minlength=2)
            right_class_counts = np.bincount(right_labels, minlength=2)

            left_purity = (left_class_counts / len(left_labels)) * 100 if len(left_labels) > 0 else [0, 0]
            right_purity = (right_class_counts / len(right_labels)) * 100 if len(right_labels) > 0 else [0, 0]

            purity_info = {
                "non_CDR group purity": left_purity.tolist(),
                "CDR group purity": right_purity.tolist()
            }

    y_pred = [0 if density > best_threshold else 1 for density in densities]

    return best_threshold, best_gini, purity_info, y_pred


# Run the combined function

CENPA_mC_threshold_chrom ={} 
for chrom in CENPA_chromosome_dict_reads_non_CDR_mC: 
    best_threshold_chrom, best_gini_chrom, purity_info_chrom, y_pred_chrom = extract_and_find_optimal_threshold(CENPA_chromosome_dict_reads_non_CDR_mC[chrom], CENPA_chromosome_dict_reads_CDR_mC[chrom] )
    CENPA_mC_threshold_chrom[chrom] = best_threshold_chrom


CENPC_mC_threshold_chrom ={} 
for chrom in CENPC_chromosome_dict_reads_non_CDR_mC: 
    best_threshold_chrom, best_gini_chrom, purity_info_chrom, y_pred_chrom = extract_and_find_optimal_threshold(CENPC_chromosome_dict_reads_non_CDR_mC[chrom], CENPC_chromosome_dict_reads_CDR_mC[chrom] )
    CENPC_mC_threshold_chrom[chrom] = best_threshold_chrom

H3K9_mC_threshold_chrom ={} 
for chrom in H3K9_chromosome_dict_reads_non_CDR_mC: 
    best_threshold_chrom, best_gini_chrom, purity_info_chrom, y_pred_chrom = extract_and_find_optimal_threshold(H3K9_chromosome_dict_reads_non_CDR_mC[chrom], H3K9_chromosome_dict_reads_CDR_mC[chrom] )
    H3K9_mC_threshold_chrom[chrom] = best_threshold_chrom

In [22]:
CENPA_CDR_dict_mA_path = '/private/groups/migalab/dan/gini_heatmap/CENPA_reads_CDR_dict_mA.csv'
CENPA_non_CDR_dict_mA_path = '/private/groups/migalab/dan/gini_heatmap/CENPA_reads_non_CDR_dict_mA.csv'

CENPA_reads_CDR_mA, CENPA_reads_non_CDR_mA = parse_csv_to_dataframe(CENPA_CDR_dict_mA_path, CENPA_non_CDR_dict_mA_path)

best_threshold_chrom, best_gini_chrom, purity_info_chrom, y_pred_chrom = extract_and_find_optimal_threshold(CENPA_reads_non_CDR_mA, CENPA_reads_CDR_mA )


In [23]:
print (best_threshold_chrom, best_gini_chrom, purity_info_chrom)

0.010980930464155464 0.2581402733764441 {'non_CDR group purity': [14.017638882231703, 85.9823611177683], 'CDR group purity': [83.54239009755983, 16.457609902440176]}


In [25]:
CENPC_CDR_dict_mA_path = '/private/groups/migalab/dan/gini_heatmap/CENPC_reads_CDR_dict_mA.csv'
CENPC_non_CDR_dict_mA_path = '/private/groups/migalab/dan/gini_heatmap/CENPC_reads_non_CDR_dict_mA.csv'

CENPC_reads_CDR_mA, CENPC_reads_non_CDR_mA = parse_csv_to_dataframe(CENPC_CDR_dict_mA_path, CENPC_non_CDR_dict_mA_path)

best_threshold_chrom, best_gini_chrom, purity_info_chrom, y_pred_chrom = extract_and_find_optimal_threshold(CENPC_reads_non_CDR_mA, CENPC_reads_CDR_mA )
print (best_threshold_chrom, best_gini_chrom, purity_info_chrom)

0.013633102908569193 0.26154258657199586 {'non_CDR group purity': [11.804161343310824, 88.19583865668918], 'CDR group purity': [80.85447592417452, 19.145524075825488]}


In [26]:
H3K9_CDR_dict_mA_path = '/private/groups/migalab/dan/gini_heatmap/H3K9_reads_CDR_dict_mA.csv'
H3K9_non_CDR_dict_mA_path = '/private/groups/migalab/dan/gini_heatmap/H3K9_reads_non_CDR_dict_mA.csv'

H3K9_reads_CDR_mA, H3K9_reads_non_CDR_mA = parse_csv_to_dataframe(H3K9_CDR_dict_mA_path, H3K9_non_CDR_dict_mA_path)

best_threshold_chrom, best_gini_chrom, purity_info_chrom, y_pred_chrom = extract_and_find_optimal_threshold(H3K9_reads_non_CDR_mA, H3K9_reads_CDR_mA )
print (best_threshold_chrom, best_gini_chrom, purity_info_chrom)

0.014652014652014652 0.3377920594569553 {'non_CDR group purity': [84.22331940751994, 15.77668059248006], 'CDR group purity': [25.37729127534167, 74.62270872465834]}


In [26]:
# Extract the true labels
y_true = [d[1] for d in combined_training_data]

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[41215     0]
 [ 1118 41726]]


In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

def prepare_data_for_logistic_regression(data):
    """
    Prepare the data for logistic regression by separating densities and labels.

    Parameters:
        data (list of tuples): Each tuple contains (density, label).

    Returns:
        X (ndarray): Feature array (densities).
        y (ndarray): Label array.
    """
    X = np.array([d[0] for d in data]).reshape(-1, 1)
    y = np.array([d[1] for d in data])
    return X, y

def train_logistic_regression(X, y):
    """
    Train a logistic regression model on the given data.

    Parameters:
        X (ndarray): Feature array (densities).
        y (ndarray): Label array.

    Returns:
        model: Trained logistic regression model.
    """
    model = LogisticRegression()
    model.fit(X, y)
    return model

def get_cutoff_threshold(model):
    """
    Calculate the cutoff threshold for logistic regression.

    Parameters:
        model: Trained logistic regression model.

    Returns:
        threshold (float): The density threshold that separates CDR from non-CDR.
    """
    intercept = model.intercept_[0]
    coefficient = model.coef_[0][0]
    threshold = -intercept / coefficient
    return threshold

def evaluate_logistic_regression(model, X, y):
    """
    Evaluate the logistic regression model and display classification report, confusion matrix, and cutoff threshold.

    Parameters:
        model: Trained logistic regression model.
        X (ndarray): Feature array (densities).
        y (ndarray): Label array.
    """
    y_pred = model.predict(X)
    report = classification_report(y, y_pred)
    conf_matrix = confusion_matrix(y, y_pred)
    cutoff_threshold = get_cutoff_threshold(model)

    print("Classification Report:\n", report)
    print("Confusion Matrix:\n", conf_matrix)
    print(f"Cutoff Threshold: {cutoff_threshold:.6f}")



# Prepare the data
X, y = prepare_data_for_logistic_regression(combined_training_data)

# Train the logistic regression model
logistic_model = train_logistic_regression(X, y)

# Evaluate the model
evaluate_logistic_regression(logistic_model, X, y)


Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.75      0.81     41215
           1       0.79      0.90      0.84     42844

    accuracy                           0.83     84059
   macro avg       0.83      0.83      0.83     84059
weighted avg       0.83      0.83      0.83     84059

Confusion Matrix:
 [[30773 10442]
 [ 4105 38739]]
Cutoff Threshold: 0.016968
